In [1]:
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import hist
from hist import Hist
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
NanoAODSchema.warn_missing_crossrefs = False

from coffea.analysis_tools import PackedSelection
from topcoffea.modules import utils
import topcoffea.modules.eft_helper as efth

In [2]:
# histogram style
hep.style.use("CMS")
params = {'axes.labelsize': 20,
          'axes.titlesize': 20,
          'legend.fontsize':20}
plt.rcParams.update(params)

In [10]:
# fname = '/project01/ndcms/hnelson2/ttbarEFT/nanoGen/TT01j2l_S1/nanoGen_101.root'
fname = '/afs/crc.nd.edu/user/h/hnelson2/ttbarEFT/input_samples/nanoGen/djr_test_nanoGen.root'

In [11]:
# Load in events from root file
events = NanoEventsFactory.from_root(
    fname,
    schemaclass=NanoAODSchema.v6,
    metadata={"dataset": "TT01j2l"},
).events()

In [12]:
wc_lst = utils.get_list_of_wc_names(fname)
print(wc_lst)

['cQd1', 'ctj1', 'cQj31', 'ctj8', 'ctd1', 'ctd8', 'ctGRe', 'ctGIm', 'cQj11', 'cQj18', 'ctu8', 'cQd8', 'ctu1', 'cQu1', 'cQj38', 'cQu8']


In [13]:
events.fields

['GenVtx',
 'GenJetAK8',
 'LHE',
 'GenJet',
 'GenMET',
 'GenDressedLepton',
 'luminosityBlock',
 'LHEReweightingWeight',
 'LHEPdfWeight',
 'GenVisTau',
 'EFTfitCoefficients',
 'LHEScaleWeight',
 'event',
 'GenPart',
 'GenIsolatedPhoton',
 'LHEWeight',
 'Generator',
 'genWeight',
 'run',
 'LHEPart',
 'PSWeight',
 'MET',
 'WCnames']

In [5]:
events.LHE.fields

['HT',
 'HTIncoming',
 'Vpt',
 'AlphaS',
 'Njets',
 'Nb',
 'Nc',
 'Nuds',
 'Nglu',
 'NpNLO',
 'NpLO']

In [9]:
events.LHE.HTIncoming

<Array [435, 117, 0, 0, ... 227, 0, 0, 72.4] type='15682 * float32[parameters={"...'>

In [9]:
events.GenJet.fields

['eta', 'mass', 'phi', 'pt', 'partonFlavour', 'hadronFlavour']

In [32]:
events.LHEPart.fields

['pt', 'eta', 'phi', 'mass', 'incomingpz', 'pdgId', 'status', 'spin']

In [31]:
events.GenPart.fields

['eta',
 'mass',
 'phi',
 'pt',
 'genPartIdxMother',
 'pdgId',
 'status',
 'statusFlags',
 'genPartIdxMotherG',
 'distinctParentIdxG',
 'childrenIdxG',
 'distinctChildrenIdxG',
 'distinctChildrenDeepIdxG']

In [15]:
events.LHEWeight.fields

['originalXWGTUP', 'DJR10', 'DJR21', 'DJR32']

In [16]:
events.LHEWeight.DJR10

<Array [216, 15.6, 10.2, 170] type='4 * float32[parameters={"__doc__": "djr 1 --...'>

In [18]:
events.LHEWeight.DJR21

<Array [130, 9, 9.25, 98.5] type='4 * float32[parameters={"__doc__": "djr 2 --> ...'>

In [19]:
events.LHEWeight.DJR32

<Array [14.4, 7.41, 6.36, 50] type='4 * float32[parameters={"__doc__": "djr 3 --...'>